# License Plate Detection and OCR Processing

## Table of Contents
1. [Frame Extraction](#frame-extraction)
2. [License Plate Detection](#license-plate-detection)
3. [Image Cropping](#image-cropping)
4. [OCR and Data Retrieval](#ocr-and-data-retrieval)

In this Jupyter notebook, we guide you through the comprehensive process of building a computer vision application with LandingLens that detects and reads license plates from videos. Each section is crafted to provide you with a conceptual understanding of the step in the process and practical code examples. The process starts with frame extraction, followed by the detection and cropping of license plates, and finally, optical character recognition (OCR) for data retrieval.

 By the end of this notebook, not only will you have a functioning license plate reader, but you'll also possess foundational knowledge and techniques that are transferable to a myriad of other computer vision applications. Whether you're aiming to recognize faces, track objects, or read text from images, the principles and methods showcased here will serve as a valuable cornerstone for your future projects.


## Installation and Setup

1. Install the `landingai` python package.
2. We prepared a video clip with license plates from different cars on a street. Download the video clip to your local drive.

The video file will be downloaded at `/tmp/license-plates.mov`

In [10]:
import subprocess, sys
subprocess.check_call([sys.executable, "-m", "pip", "install", "ipywidgets"])
# Enable widgets in classic notebooks (skip in JupyterLab >=3)
subprocess.check_call([sys.executable, "-m", "jupyter", "nbextension", "enable",
                       "--py", "widgetsnbextension"])

0

<a name="frame-extraction"></a>
## Frame Extraction

In this section, we'll be extracting frames from a given video file. By reading the video frame-by-frame, we aim to save specific frames based on a set interval (e.g., every 100th frame) for further processing. This approach helps in reducing the computational load by processing only a subset of the frames instead of every single one.


In [12]:
import io, os, tempfile
from IPython.display import display, Video
import ipywidgets as widgets

from landingai.pipeline.image_source import VideoFile
from landingai.predict import Predictor, OcrPredictor
from landingai import visualize
from landingai.postprocess import crop

In [13]:
upload = widgets.FileUpload(accept='video/*', multiple=False)
output = widgets.Output()

def on_upload(change):
    output.clear_output()
    if not upload.value:
        return
    for fname, meta in upload.value.items():
        # Save to a temp file
        suffix = os.path.splitext(fname)[1]
        tmp = tempfile.NamedTemporaryFile(delete=False, suffix=suffix)
        tmp.write(meta['content'])
        tmp.flush()
        tmp_path = tmp.name

        # Display uploaded video
        with output:
            print(f"Saved upload as {tmp_path}")
            display(Video(tmp_path, width=512))

          from landingai.pipeline.image_source import VideoFile
video_source = VideoFile(tmp_path, samples_per_second=1)
frames = [f.image for f in video_source]
from landingai.predict import Predictor
from landingai import visualize

predictor = Predictor(model_endpoint, api_key=api_key_lp)
bounding_boxes, overlayed_frames = [], []
for frame in frames:
    pred = predictor.predict(frame)
    overlayed_frames.append(visualize.overlay_predictions(pred, frame))
    bounding_boxes.append(pred)
from landingai.postprocess import crop
cropped_imgs = []
for frame, bboxes in zip(frames, bounding_boxes):
    cropped_imgs.append(crop(bboxes, frame))
from landingai.predict import OcrPredictor
ocr_predictor = OcrPredictor(api_key=api_key_ocr)
ocr_results, overlayed_ocr = [], []
for plate_group in cropped_imgs:
    for plate_img in plate_group:
        o = ocr_predictor.predict(plate_img)
        overlayed_ocr.append(visualize.overlay_predictions(o, plate_img))
        ocr_results.append(o)

            # 3a. Extract frames (1 sample/sec)
            video_source = VideoFile(tmp_path, samples_per_second=1)
            frames = [f.image for f in video_source]
            print(f"Extracted {len(frames)} frames")

            # 3b. Run license‐plate detection
            api_key_lp = "land_sk_aMemWbpd41yXnQ0tXvZMh59ISgRuKNRKjJEIUHnkiH32NBJAwf"
            model_endpoint = "e001c156-5de0-43f3-9991-f19699b31202"
            lp_predictor = Predictor(model_endpoint, api_key=api_key_lp)

            bboxes, overlayed = [], []
            for frame in frames:
                pred = lp_predictor.predict(frame)
                overlayed.append(visualize.overlay_predictions(pred, frame))
                bboxes.append(pred)

            # Show first few detection overlays
            print("Detection overlays:")
            for ov in overlayed[:5]:
                display(ov)

            # 3c. Crop plates
            cropped_imgs = []
            for frame, boxes in zip(frames, bboxes):
                cropped_imgs.append(crop(boxes, frame))
            print(f"Cropped {sum(len(c) for c in cropped_imgs)} plate images")

            # Show a few crops
            print("Plate crops:")
            for crops in cropped_imgs[:3]:
                for c in crops:
                    display(c)

            # 3d. OCR on each crop
            api_key_ocr = "land_sk_WVYwP00xA3iXely2vuar6YUDZ3MJT9yLX6oW5noUkwICzYLiDV"
            ocr = OcrPredictor(api_key=api_key_ocr)

            print("OCR results:")
            for crops in cropped_imgs:
                for plate_img in crops:
                    o = ocr.predict(plate_img)
                    ov = visualize.overlay_predictions(o, plate_img)
                    display(ov)
                    for txt in o:
                        print(txt.text)

upload.observe(on_upload, names='value')

# 4. Display widget & output
display(widgets.VBox([
    widgets.HTML(value="<h4>Upload a video file for license-plate OCR:</h4>"),
    upload,
    output
]))

<a name="license-plate-detection"></a>

## License Plate Detection
Once we have our frames, the next step is to detect license plates within these frames. We'll be using a predefined API to help us detect the bounding boxes around the license plates. The results will be overlayed on the frames to visualize the detections.